In [1]:
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

In [2]:
#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file('test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [3]:
#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
text_train = open(train_file,'rb').read().decode(encoding='utf-8')
text_test = open(test_file,'rb').read().decode(encoding='utf-8')

In [4]:
print(text_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
# train y값 label을 먼저 뽑는다. 
a=text_train[:500].split('\n')[1:]
#줄마다 나눠서 a에 넣는다.
print(a[:5])
print(a[3].count('\t'))
# a[5]


['9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0', '3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1', '10265843\t너무재밓었다그래서보는것을추천한다\t0', '9045019\t교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정\t0', '6483659\t사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다\t1']
2


In [6]:
# '9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0'
# '9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'
a[0].split('\t')[2]  # 감상평의 라벨 0= 부정,  1= 긍정

'0'

In [7]:
# for row in text_train.split('\n')[1:]: 
#     print(row) #여기 한줄 뽑아서 
#     print(row.split('\t')[2]) #\t 로 나눠서 2번째 원소인 라벨을 뽑는다
#     print( int(row.split('\t')[2]) ) # int() 해서 정수형으로 바꿔준다.

In [8]:
# 데이터 정제 

print(text_test[:200])

#각 줄을 \n으로 분리하겠다.
#헤더를 제외하겠다. 

#한줄마다 \t로 분할해서 문장과 점수로 나눈다
#감상평은 \t로 구분되어있음
# [1:]으로 헤더제거

#감상평의 평점만 뽑아서 정수화 행렬로 만들어줬다.
y_train= np.array([ [int(row.split('\t')[2])] for row in text_train.split('\n')[1:] if row.count('\t')>0])
y_test= np.array([ [int(row.split('\t')[2])] for row in text_test.split('\n')[1:] if row.count('\t')>0])
print(y_train.shape) # (150000개의 평점 , 1개의 열)


id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심
(150000, 1)


In [ ]:
text_train.split('\n')  #한줄씩 분리하고
text_train.split('\n')[1:]  #헤더 처리
#row.count('\t')>0   평점이 존재하면
# row.split('\t')[1]  감상평만 추출

In [10]:
#X 값 추출 = 리뷰문장
x_train = [ row.split('\t')[1] for row in text_train.split('\n')[1:] if row .count('\t')>0  ]
x_train[:3]


['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다']

In [11]:
#!pip install konlpy

In [12]:
#kkma로도 한번 해보기
from konlpy.tag import Kkma

# x_train 를 넣어본다.
# Kkma().sentences( x_train[0] )

#띄어쓰기기준으로 나눠서 토큰화
x_train[0].split(' ')



['아', '더빙..', '진짜', '짜증나네요', '목소리']

In [ ]:
#위에서 추출한 문장을 토큰화해볼것이다. => OKT 방법을 사용할것이다.
from konlpy.tag import Okt
#konlpy 패키지에있는 tag 모듈에 Okt 메서드를 불러온다.

x_train_tokenized = []
for i in x_train[:20000]: #너무오래걸려서 리뷰 2만개만
    x_train_tokenized.append( Okt().morphs( i ) ) 

#시간이 오래걸리네 #GPU로 5분정도
x_train_tokenized


In [14]:
# 토큰화된 학습 데이터
# x_train_tokenized[:100]
# 나눠진 토큰들을 임베딩할것이다. 
len(x_train_tokenized)

total_len = 0
for i in x_train_tokenized:
    total_len += len(i)
print(total_len)  # 토큰 2,156,163 개  => 은, 는, 이, 가 이게 많이 나오면 중요 키워드인가? 
# 그렇지않다. => 그럼 제거한다


289293


만약 의문문 대해서 커스텀을하고싶으면 


In [15]:
# Okt().morphs( '오늘 뭐먹었어?' )
# Okt().morphs( '오늘 뭐했어?' ) #의문문에 대한 구분이 불가능 해보인다. 

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words= 5000 ) #빈도가 높은것 순서대로 2만개만 다시 토큰화한다.

#빈도수 기준으로 사전을 만든다.  단어사전 =단어순서대로 숫자를 부여한다(인덱싱)
tokenizer.fit_on_texts( x_train_tokenized )
 

# 인덱싱된 데이터들을 시계열데이터로 변환한다. == 수치화 == 정수 인코딩이라한다.  
sequences = tokenizer.texts_to_sequences( x_train_tokenized )

#이렇게 숫자로 바뀌면 이제 컴퓨터가 처리할수있는 데이터가 된다.
sequences


In [ ]:
sequences

In [18]:
# 패딩 == 공간을 준다.
from tensorflow.keras.preprocessing.sequence import pad_sequences

#가장 긴 문장을 기준으로 다른문장들도 길이를 맞춰준다.  늘어난 공간은 0 으로한다.
X_train = pad_sequences(sequences,padding='post')  #학습데이터 완성
# y_train

# [1] => 세개 공간을 만들고 싶다. = pre앞에[0 0 1], post뒤에[1 0 0]

In [23]:
print(X_train[0]) #문장 하나의 시퀀스데이터
print(X_train[0].shape)
# (150000, 93) 15만개 리뷰, 가장긴 문장의 토큰개수 == 93 == 패딩된 값

[ 62 537   5  23 804   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
(66,)


In [24]:
X_train

array([[  62,  537,    5, ...,    0,    0,    0],
       [ 822,    9,  391, ...,    0,    0,    0],
       [ 371, 2243, 4778, ...,    0,    0,    0],
       ...,
       [ 290,  157,   43, ...,    0,    0,    0],
       [  36,    6,   68, ...,    0,    0,    0],
       [ 469,  919,  194, ...,    0,    0,    0]], dtype=int32)

In [36]:
# 딥러닝 모델
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
# from tensorflow.keras.

#딥러닝기반 언어(텍스트정제 => 숫자로된 시계열데이터로 들어온다 )처리모델 

model = Sequential()

#들어온 시퀀스데이터를 벡터로 변환한다.5000==단어사전크기, 600 차원은 클수록 좋다. 인풋랭스는 시퀀스데이터의 길이이다.
model.add( Embedding( 5000, 1000, input_length = 66)) #입력층

model.add( LSTM( units = 1000) )   #은닉층,  units: 뉴런갯수 많을수록 좋다.

#0부정,1긍정
model.add( Dense( 2, activation='softmax' ))  #출력층
#activation='softmax'이면 자동으로 loss='sparse_categorical_crossentropy'이다.


# compile: ex) c언어 소스코드 => 컴터가 못알아들음 => 그럼면 알아들을수있게 컴파일해준다.
#컴파일  , optimazer, loss ,metrics
# sparse_categorical_crossentropy는 categorical_crossentropy와 동일하지만, 정수 형태의 레이블을 사용
model.compile(optimizer='adam',  loss='sparse_categorical_crossentropy',
              metrics=['acc'])



In [37]:
model.summary()
# params == 파라미터 == 매개변수 == 가중치w랑 편향b의 개수 

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 66, 1000)          5000000   
                                                                 
 lstm_4 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense_4 (Dense)             (None, 2)                 2002      
                                                                 
Total params: 13,006,002
Trainable params: 13,006,002
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#모델을 학습시킨다.=> 새로운 데이터가오면 판단을 할수있다.
model.fit( X_train, y_train, epochs=5, batch_size = 500 , validation_split = 0.2)
#X =>   y
#리뷰글 => 점수
#피쳐  => 타겟
#독립변수 => 종속변수
#학습데이터 => 라벨링

#모델링
#모델에서 여러가지 바꿔봐도 성능이 개선되지않는다. => 전처리 과정을 바꿔봐야한다.

In [ ]:
# 테스트 해보기
a='영화 완전 노잼입니다.'  # 0이 나오면된다.

b='대박 개꿀잼임ㄹㅇ'    # 1이 나오면된다.

c='나쁘지도 않고 좋지도 않았다.'  #수빈님 예측=> 부정  이런거는 틀릴 수 밖에 없다. => 왜냐면 가르친적이 없다.
# => 라벨 2를 추가해서 학습을 시킨다.

